# O projeto
![Imagem do sistema](docs/system_proposal.png)

In [1]:
# feito por arthur lorencini
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [2]:
# Definindo os os caminhos
raw_dataset_path = Path("data/raw/")
wind_dataset_path = raw_dataset_path / 'wind/VentoCurvaPotencia.txt'
consuption_dataset_path = raw_dataset_path / 'consumption' / 'Brava_consumo.xlsx'
cases_dataset_path = raw_dataset_path / 'cases' / 'cases.csv'

In [3]:
# lendo o arquivo de texto que contém a quantidade de vento
df_wind = pd.read_csv(wind_dataset_path,sep='\t',header=None)
df_wind.columns = ['velocity','2','3','4']
# O que os números significam:
# Os números são para indicar 
# 2:ntk150, 3:ntk300, 4:acsa225
print(df_wind.head())
print("O dataset deve conter 4 colunas e 8760 linhas")
print(df_wind.shape)

   velocity     2     3     4
0       6.5  39.5  44.4  41.8
1       5.6  24.4   0.0  26.5
2       5.3  18.5   0.0  21.2
3       2.4   0.0   0.0   0.0
4       3.1   0.0   0.0   0.0
O dataset deve conter 4 colunas e 8760 linhas
(8760, 4)


In [4]:
# pegando os dados de consumo/demanda da ilha
# O dataset possui umas linhas que não vão agregar em nada
# Vamos apenas pegar as linhas
df_cons = pd.read_excel(consuption_dataset_path,sheet_name='Carga2010-2037',skiprows=1,nrows=8760)
if df_cons.shape[0] != 8760:
    raise ValueError("O dataset de consumo está com o tamanho errado")
df_cons.head()


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,...,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,252.0,259.56,267.3468,275.367204,283.628220,292.137067,277.959276,286.298054,294.886996,303.733606,...,396.303464,408.192568,420.438345,433.051495,446.043040,459.424331,473.207061,487.403273,502.025371,517.086132
1,238.0,245.14,252.4942,260.069026,267.871097,275.907230,262.517094,270.392607,278.504385,286.859517,...,374.286605,385.515203,397.080659,408.993079,421.262871,433.900757,446.917780,460.325313,474.135073,488.359125
2,228.0,234.84,241.8852,249.141756,256.616009,264.314489,251.486964,259.031573,266.802520,274.806596,...,358.560277,369.317085,380.396598,391.808496,403.562750,415.669633,428.139722,440.983914,454.213431,467.839834
3,225.0,231.75,238.7025,245.863575,253.239482,260.836667,248.177925,255.623263,263.291961,271.190719,...,353.842378,364.457650,375.391379,386.653121,398.252714,410.200296,422.506305,435.181494,448.236939,461.684047
4,220.0,226.60,233.3980,240.399940,247.611938,255.040296,242.662860,249.942746,257.441028,265.164259,...,345.979214,356.358591,367.049349,378.060829,389.402654,401.084734,413.117276,425.510794,438.276118,451.424401


Cada ano mostra o consumo da ilha de Brava, se eu não me engano. Cada ano possui 8760 horas

In [5]:
# Definindo constantes
# Eficiência média da bomba
avg_pump_eff = 0.86 # nb
# Eficiência média da turbina hidráulica (eu acho)
avg_turbine_eff = 0.86 # nt
# Eficiência média da conversão da potência eólica
avg_conversion_eff = 0.97 # nw

# O mínimo de potência de entrega?
min_power_delivery = 256*0.5 # PDmin


In [6]:
# Definindo a matrix de resultados
df_result = pd.DataFrame({})


In [14]:
# Definindo os casos (cenários simulados)
# Esse csv eu criei para usar como banco de dados
df_cases = pd.read_csv(cases_dataset_path)
df_cases.head()

,aero_power_kW,number_of_aero,aero_id,reservatory_volume_m3
0,150,2,2,1500
1,300,1,3,1500
2,150,3,2,2250
3,225,2,4,2250
4,225,3,4,3400


aero_id se refere o id lá do dataset do vento.

In [15]:
def resevartory_volume_to_energy(volume_m3:float):
    """recebe em m3 e devolve em kWh
     o quanto de energia um reservatório pode armazenar
     110 é a altura do reservatório [m]
     9.8 é a acc da gravidade [m/s^2]
     Tá certo mesmo essa equação?
     E = mgh
     E = d*vol*g*h
     d é realmente 1/3600?
     """
    return volume_m3/3600*9.8*110

In [16]:
# Aqui vamos calcular o quanto que cada combinação pode nos dar de energia inicial
df_cases['reservatory_stored_energy_kWh'] = resevartory_volume_to_energy(df_cases.reservatory_volume_m3)
df_cases.head()

,aero_power_kW,number_of_aero,aero_id,reservatory_volume_m3,reservatory_stored_energy_kWh
0,150,2,2,1500,449.166667
1,300,1,3,1500,449.166667
2,150,3,2,2250,673.750000
3,225,2,4,2250,673.750000
4,225,3,4,3400,1018.111111


![Algo](docs/algo.png)

In [10]:
injected_power_values = []
for idx, case_row in df_cases.iterrows():
    aero_power = case_row['aero_power']
    number_of_aero = case_row['number_of_aero']
    aero_id = str(int(case_row['aero_id']))
    max_stored_energy = case_row['reservatory_stored_energy']
    initial_stored_energy = max_stored_energy/2

    # eff = out/in → eff = elect/wind → wind = elect*eff
    total_wind_turbine_power = number_of_aero * avg_conversion_eff * df_wind[aero_id] 
    nominal_turbine_power = total_wind_turbine_power.mean()

    for year in df_cons.columns:
        for hour_idx, consumption in enumerate(df_cons[year]):
            if total_wind_turbine_power[hour_idx] > consumption*0.35:
                injected_power = consumption*0.35 # TODO: Armazenar em uma matriz
                injected_power_values.append(injected_power)
                v1 = (total_wind_turbine_power[hour_idx] - injected_power)*avg_pump_eff
                v2 = max_stored_energy - initial_stored_energy
                if v1 < v2:
                    # Verifica se o reservatório tem capacidade para 
                    # absorver mais água bombeada
                    # pb
                    pump_power = total_wind_turbine_power[hour_idx] - injected_power
                    initial_stored_energy += v1
                else:
                    # cuidado com as unidades
                    pump_power = v2/avg_pump_eff
                    initial_stored_energy = max_stored_energy
                    rejected_energy = total_wind_turbine_power[hour_idx] - injected_power
                    rejected_energy -= (max_stored_energy - initial_stored_energy)/avg_pump_eff
            else:
                injected_power = total_wind_turbine_power[hour_idx]

            if consumption - injected_power <= initial_stored_energy*avg_turbine_eff:
                cond1 = consumption - injected_power > nominal_turbine_power
                cond2 = consumption - injected_power <= nominal_turbine_power + min_power_delivery
                if consumption - injected_power <= nominal_turbine_power:
                    pass 
                    # TODO:
                elif cond1 and cond2:
                    diesel_generation_power = min_power_delivery
                    hidraulic_turbine_power = consumption - injected_power - diesel_generation_power
                    initial_stored_energy -= hidraulic_turbine_power/avg_turbine_eff
                elif cond3:
                    pass
            elif cond4:
                if cond5 and cond6:
                    pass
                else:
                    pass
                



NameError: name 'cond4' is not defined